In [92]:
from utils import *
from pysradb.sraweb import SRAweb

todo: change paths to relative ones in project-tobamo

In [93]:
# import data
non_celular_path = '/home/tobamo/analize/model-tobamo/data/snakemake_output/test_contigs_2024-04-09_non_cellular.fasta'
gt = pd.read_excel('/home/tobamo/analize/model-tobamo/notebooks/MODEL4/data/ground_truth_20250630.xlsx')
template_cols = pd.read_excel('/home/tobamo/analize/model-tobamo/notebooks/MODEL4/data/Tobamo - tabela za tobamo kontige - kategorije.xlsx').columns
sra_metadata_template_cols = pd.read_excel('/home/tobamo/analize/model-tobamo/notebooks/MODEL4/data/mapping_sra_metadata.xlsx')

# SRRs to remove (after manual curation on 4.7.2025 (WA correpondence)) due to replicated uploads of same studies (Panonychus citri (SRP187337, SRP188804) in pa VIZIONS (ERP108694, ERP006046))
SRRs_to_remove = ['ERR2737479', 'SRR8749694', 'SRR8749695', 'SRR8749695', 'SRR8749693']

# Create an empty DataFrame with the template columns
df = pd.DataFrame(columns=template_cols)

# parse fasta
records = list(SeqIO.parse(non_celular_path, 'fasta'))

# strip headers
gt.columns = gt.columns.str.strip()

# filter records based on ground truth
contig_names = gt.contig_name.unique()
records = [r for r in records if r.id in contig_names]

In [94]:
# start filling df columns
df['contig_id'] = [rec.id for rec in records]
df['sequence'] = [str(rec.seq) for rec in records]
df['corresponding_srr'] = df['contig_id'].str.split('_').str[-1]

# remove SRRs that are in the SRRs_to_remove list
df = df.copy()
df = df[~df['corresponding_srr'].isin(SRRs_to_remove)]

df['assembler'] = np.where(df['contig_id'].str.contains('NODE'), 'spades', 'megahit')
df['contig_length'] = [len(seq) for seq in df['sequence']]

# make mapper, add ground truth category
gt_category_mapper = gt.set_index('contig_name')['category'].to_dict()
df['ground_truth_category'] = df['contig_id'].map(gt_category_mapper)

df['known_or_potentially_novel_tobamovirus'] = np.where(df['ground_truth_category'].isin([1,2,8]), True, False)

import SRA metadata

In [96]:
## RUN ONLY ONCE
# # Get unique SRR values
# srr_list = df['Corresponding SRR'].unique().tolist()

# # Initialize SRAweb
# db = SRAweb()

# # Fetch metadata for all SRRs
# metadata = db.sra_metadata(srr_list, detailed=True)

# # save
# metadata.to_csv('results/sra_metadata.csv', index=False)

In [97]:
# import metadata
metadata = pd.read_csv('results/sra_metadata.csv')

select columns for parsing

In [98]:
# create sra_metadata_columnames_mapper
sra_mapper = sra_metadata_template_cols.dropna().set_index('run_accession')['corresponding_srr'].to_dict()

# keep selected columns
metadata_sub = metadata[['run_accession', 'study_accession', 'study_title', 'organism_name', 'collection_date', 'geo_loc_name', 'insdc center name']].drop_duplicates()
metadata_sub.rename(columns={'run_accession': 'corresponding_srr'}, inplace=True)

In [100]:
# Create mapping dictionaries for each metadata column
col_map = {
    'collection_date': dict(zip(metadata_sub['corresponding_srr'], metadata_sub['collection_date'])),
    'study_accession': dict(zip(metadata_sub['corresponding_srr'], metadata_sub['study_accession'])),
    'study_title': dict(zip(metadata_sub['corresponding_srr'], metadata_sub['study_title'])),
    'organism_name': dict(zip(metadata_sub['corresponding_srr'], metadata_sub['organism_name'])),
    'country': dict(zip(metadata_sub['corresponding_srr'], metadata_sub['geo_loc_name'])), 
    'submitter': dict(zip(metadata_sub['corresponding_srr'], metadata_sub['insdc center name'])),
}

# Map each metadata column to df using the corresponding_srr
for col, mapper in col_map.items():
    df[col] = df['corresponding_srr'].map(mapper)

In [101]:
date = datetime.now().strftime('%Y-%m-%d')
df.to_csv(f'results/{date}_tobamo_novel_contigs_table.csv', index=False)

In [57]:
date = '2025-07-07'
df = pd.read_csv(f'results/{date}_tobamo_novel_contigs_table.csv')